In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np


In [ ]:
## load the trained modek,scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler

with open('onehot_encoder_geo.pk1','rb') as file:
   onehot_encoder_geo= pickle.load(file)
with open('label_encoder_gender.pk1','rb') as file:
    label_encoder_gender= pickle.load(file)

with open('scaler.pk1','rb') as file:
    scaler=pickle.load(file)

In [21]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 30,
    'Balance':6000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}






In [15]:
geo_encoder=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoder_df =pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

d:\Opalina_project\python\pyenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
input_df =pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,30,6000,2,1,1,50000


In [23]:
## encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,30,6000,2,1,1,50000


In [24]:
## cocatination one hot encoded
input_df =pd.concat([input_df.drop("Geography",axis=1),geo_encoder_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,30,6000,2,1,1,50000,1.0,0.0,0.0


In [25]:
## Scalling the data
input_scaled =scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359,  8.67426486, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [26]:
## predict  churn
prediction =model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 192ms/step


array([[0.01764265]], dtype=float32)

In [30]:
prediction_prob =prediction[0][0]
prediction_prob

0.017642645

In [31]:
if prediction_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
